In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math
import requests
from io import BytesIO
from datetime import datetime

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from PIL import Image

from resources.tier_chart import tier_chart
from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_team_info, get_pbp_data
from resources.team_stats import get_team_stats
from resources.player_stats import get_player_stats

pio.templates['nfl_template'] = nfl_template

In [2]:
''' Params / Constants '''

## Params ##
WEEK = 10
EXPORT = False

## Constants ##

LEAGUE_LOGO = 'https://raw.githubusercontent.com/nflverse/nflverse-pbp/master/NFL.png'
PLOT_BUFFER = 0.05

In [3]:
''' Import Data '''

# Import
team_data = get_team_info()
pbp_data = get_pbp_data(years=[2025]).copy()

# Filter to current week
league_data = pbp_data.loc[pbp_data['week'] <= WEEK, :].copy()

week_data = pbp_data.loc[pbp_data['week'] == WEEK, :].copy()

print(team_data.shape)
print(team_data.head().to_string())
print(league_data.shape)
print(league_data.tail().to_string())
print(week_data.shape)
print(week_data.head().to_string())

2025 done.
Downcasting floats.
(32, 15)
              team_name  team_id  team_nick team_conf team_division team_color team_color2 team_color3 team_color4                                                                                                   team_logo_wikipedia                                          team_logo_espn                                                          team_wordmark                                         team_conference_logo                                                        team_league_logo                                                          team_logo_squared
team                                                                                                                                                                                                                                                                                                                                                                                                    

In [4]:
''' Get Stats '''

## Offense ##

# Overall
team_offense = get_team_stats(league_data, unit='offense')
team_offense = team_offense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(team_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())

# Week
week_offense = get_team_stats(week_data, unit='offense')
week_offense = week_offense.merge(team_data[['team_color', 'team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(week_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())


## Defense ##

# Overall
team_defense = get_team_stats(league_data, unit='defense')
team_defense = team_defense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(team_defense.sort_values(by='EPA / Play', ascending=True).head().to_string())

# Week
week_defense = get_team_stats(week_data, unit='defense')
week_defense = week_defense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(week_defense.sort_values(by='EPA / Play', ascending=True).head().to_string())

         Games  Plays  OnSchedulePlays   TDs  FirstDowns  ThirdDownAtts  ThirdDownConvs  RushAttempts  RushYards  RushTDs  Rush1Ds  DesignedRushPlays  DesignedRushAttempts  DesignedRushYards  QBScrambles  ScrambleYards  DesignedPassPlays  Dropbacks  PassCompletions  PassAttempts  PassYards  PassTDs  Pass1Ds  Sacks  SackYards  INTs  TFLs  Fumbles  Penalties  PenaltyYards  Penalty1Ds  Drives  Total Yards  Turnovers  PlaysAdv  PassPlays  RushPlays         EPA    RushEPA    PassEPA  Successes  RushSuccesses  PassSuccesses       WPA   RushWPA   PassWPA  EPA / Play  Rush EPA / Play  Pass EPA / Play  Success Rate  Rush Success Rate  Pass Success Rate                                     team_logo_espn                                                          team_wordmark
posteam                                                                                                                                                                                                                           

In [5]:
''' Get Games '''

## Games ##
games = nfl.import_schedules(years=[2025]).copy()
games = games.loc[(games['week'] == WEEK) & ~(games['result'].isna()), 
                  ['home_team', 'home_score', 'away_team', 'away_score', 'result']].copy()

games['game result'] = games['away_team'] + ' ' + games['away_score'].astype(int).astype(str) + ' - ' + games['home_score'].astype(int).astype(str) + ' ' + games['home_team']
games['winning_team'] = np.where(games['result'] > 0, games['home_team'], games['away_team'])
games['losing_team'] = np.where(games['result'] > 0, games['away_team'], games['home_team'])

## Add Winner and Loser Stats
games = games.merge(week_offense, left_on='winning_team', right_index=True)
games = games.merge(week_offense, left_on='losing_team', right_index=True, suffixes=('_winner', '_loser'))

games['Net Success Rate'] = games['Success Rate_winner'] - games['Success Rate_loser']
games['Net EPA'] = games['EPA_winner'] - games['EPA_loser']
games['Net EPA / Play'] = games['EPA / Play_winner'] - games['EPA / Play_loser']

# games['winning_color'] = np.where(games['result'] > 0, games['team_color_winner'], games['team_color_loser'])

games = games.sort_values(by='Net Success Rate', ascending=False)

print(games.to_string())

     home_team  home_score away_team  away_score  result      game result winning_team losing_team  Games_winner  Plays_winner  OnSchedulePlays_winner  TDs_winner  FirstDowns_winner  ThirdDownAtts_winner  ThirdDownConvs_winner  RushAttempts_winner  RushYards_winner  RushTDs_winner  Rush1Ds_winner  DesignedRushPlays_winner  DesignedRushAttempts_winner  DesignedRushYards_winner  QBScrambles_winner  ScrambleYards_winner  DesignedPassPlays_winner  Dropbacks_winner  PassCompletions_winner  PassAttempts_winner  PassYards_winner  PassTDs_winner  Pass1Ds_winner  Sacks_winner  SackYards_winner  INTs_winner  TFLs_winner  Fumbles_winner  Penalties_winner  PenaltyYards_winner  Penalty1Ds_winner  Drives_winner  Total Yards_winner  Turnovers_winner  PlaysAdv_winner  PassPlays_winner  RushPlays_winner  EPA_winner  RushEPA_winner  PassEPA_winner  Successes_winner  RushSuccesses_winner  PassSuccesses_winner  WPA_winner  RushWPA_winner  PassWPA_winner  EPA / Play_winner  Rush EPA / Play_winner  Pass EPA

In [52]:
''' Week Team Performances '''

## Data
team_df = team_offense[['EPA / Play']].merge(team_defense[['EPA / Play', 'team_logo_espn']], left_index=True, right_index=True)
team_df = team_df.rename(columns={
    'EPA / Play_x': 'Offensive EPA / Play',
    'EPA / Play_y': 'Defensive EPA / Play',
})

week_df = week_offense[['EPA / Play']].merge(week_defense[['EPA / Play', 'team_logo_espn']], left_index=True, right_index=True)
week_df = week_df.rename(columns={
    'EPA / Play_x': 'Offensive EPA / Play',
    'EPA / Play_y': 'Defensive EPA / Play',
})

total_df = pd.concat([team_df, week_df])
print(total_df.shape)
print(total_df.head().to_string())

## Chart
fig = tier_chart(
    data_frame=total_df,
    x_col='Offensive EPA / Play',
    y_col='Defensive EPA / Play',
    logos_col='team_logo_espn',
    title=f'<b>Are We Really That Bad?</b><br><sup>Week {WEEK} performance (full color) compared to season performance (transparent)</sup>',
    n_tiers=0,
    x_reversed=False,
    y_reversed=True
)

fig.update_xaxes(
    tickformat='0.2f',
    linecolor='#f0f0f0', mirror=True
)
fig.update_yaxes(
    tickformat='0.2f',
    linecolor='#f0f0f0', mirror=True
)
fig.update_layout_images(sizey=0.1, opacity=0.9)
fig.update_annotations(y=-.11)
fig.update_layout(
    width=700, height=500,
    # paper_bgcolor='#fafafa', plot_bgcolor='white',
)

# Make year long opaque
for i in range(32):
    fig.layout.images[i].update(opacity=0.5, sizey=0.07)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Are We Really That Bad? - Week {WEEK}.png',scale=6, width=700, height=500)

(58, 3)
     Offensive EPA / Play  Defensive EPA / Play                                          team_logo_espn
ARI                 0.000                 0.021       https://a.espncdn.com/i/teamlogos/nfl/500/ari.png
ATL                -0.021                 0.027       https://a.espncdn.com/i/teamlogos/nfl/500/atl.png
BAL                 0.025                 0.087       https://a.espncdn.com/i/teamlogos/nfl/500/bal.png
BUF                 0.152                 0.026       https://a.espncdn.com/i/teamlogos/nfl/500/buf.png
CAR                -0.031                 0.065  https://a.espncdn.com/i/teamlogos/nfl/500-dark/car.png
y = 0.26954x + 0.012340692113715308
x_range = np.float64(0.884)
y_range = np.float64(0.884)
slope = np.float64(1.0)
1
0


In [ ]:
''' Did We Really Get Beat That Bad? '''

for net_col in ['Net Success Rate', 'Net EPA']:
    games = games.sort_values(by=net_col, ascending=False)
    
    x = games['game result'].to_numpy()
    y = games[net_col].to_numpy()

    colors = games['team_color_winner'].tolist()
    winner_logos = games['team_logo_espn_winner'].tolist()
    loser_logos = games['team_logo_espn_loser'].tolist()

    # Column
    column = px.bar(
        x=x,
        y=y,
    )

    # Create Figure
    fig = go.Figure()

    for trace in column.data:
        fig.add_trace(trace)

    # Loser Logos
    logo_size = 0.5 if net_col == 'Net Success Rate' else 3

    for i in range(len(x)):
        fig.add_layout_image(
            source=loser_logos[i],  # The loaded image
            xref="x",    # Reference x-coordinates to the x-axis
            yref="y",    # Reference y-coordinates to the y-axis
            x=x[i], # X-coordinate of the image's center
            y=0, # Y-coordinate of the image's center
            sizex=logo_size,   # Width of the image in data units
            sizey=logo_size,   # Height of the image in data units
            xanchor="center", # Anchor the image by its center horizontally
            yanchor="middle", # Anchor the image by its middle vertically
            layer="above", # Place image above other plot elements
            opacity=0.8
        )

    # Winner logos
    for i in range(len(x)):
        fig.add_layout_image(
            source=winner_logos[i],  # The loaded image
            xref="x",    # Reference x-coordinates to the x-axis
            yref="y",    # Reference y-coordinates to the y-axis
            x=x[i], # X-coordinate of the image's center
            y=y[i], # Y-coordinate of the image's center
            sizex=logo_size,   # Width of the image in data units
            sizey=logo_size,   # Height of the image in data units
            xanchor="center", # Anchor the image by its center horizontally
            yanchor="middle", # Anchor the image by its middle vertically
            layer="above", # Place image above other plot elements
            opacity=0.8
        )

    # Format
    fig.update_yaxes(
        title=net_col if net_col == 'Net Success Rate' else f'Expected Margin (Net EPA)',
        tickformat='.0%' if net_col == 'Net Success Rate' else '.0f',
        linecolor='#f0f0f0', mirror=True,
    )
    fig.update_xaxes(
        tickfont=dict(size=8),
        tickangle=-25,
        linecolor='#f0f0f0', mirror=True
    )
    fig.update_traces(marker=dict(opacity=0.7, color=colors, line=dict(width=2, color='#323232')))
    fig.update_layout(
        template='nfl_template',
        title=f'<b>Did We Really Get Beat That Bad?</b><br><sup>{net_col}, Week {WEEK}</sup>',
        height=500, width=900,
        margin_pad=0,
        margin=dict(t=50,b=60,l=40)
    )

    # Credits
    fig.add_annotation(
        text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
        showarrow=False,
        xref='paper',
        yref='paper',
        y=-0.15, 
        x=1,
        align='right'
    )

    fig.show()

    # Export
    if EXPORT: pio.write_image(fig, f'Did We Really Get Beat That Bad? - {net_col} - Week {WEEK}.png',
                            scale=6, width=900, height=500)

In [50]:
''' Did We Really Get Beat That Bad? Subplots'''

## Data ##
games = games.sort_values(by='Net Success Rate', ascending=False)

x = games['game result'].to_numpy()

colors = games['team_color_winner'].tolist()
winner_logos = games['team_logo_espn_winner'].tolist()
loser_logos = games['team_logo_espn_loser'].tolist()

## Success Rate
sr_y = games['Net Success Rate'].to_numpy()

net_sr_column = px.bar(
    x=x,
    y=sr_y,
)

## EPA
epa_y = games['Net EPA'].to_numpy()

net_epa_column = px.bar(
    x=x,
    y=epa_y,
)


# Create Figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=.05)

for trace in net_sr_column.data:
    fig.add_trace(
        trace,
        row=1, col=1
    )

for trace in net_epa_column.data:
    fig.add_trace(
        trace,
        row=2, col=1
    )

fig.update_traces(marker=dict(opacity=0.7, color=colors, line=dict(width=2, color='#323232')))

# Loser Logos
logo_size = 0.5 if net_col == 'Net Success Rate' else 3

for i in range(len(x)):
    fig.add_layout_image(
        source=loser_logos[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=0, # Y-coordinate of the image's center
        sizex=0.45,   # Width of the image in data units
        sizey=0.45,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.8
    )
for i in range(len(x)):
    fig.add_layout_image(
        source=loser_logos[i],  # The loaded image
        xref="x2",    # Reference x-coordinates to the x-axis
        yref="y2",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=0, # Y-coordinate of the image's center
        sizex=4,   # Width of the image in data units
        sizey=4,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.8
    )

# Winner logos
for i in range(len(x)):
    fig.add_layout_image(
        source=winner_logos[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=sr_y[i], # Y-coordinate of the image's center
        sizex=0.45,   # Width of the image in data units
        sizey=0.45,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.8
    )
    fig.add_layout_image(
        source=winner_logos[i],  # The loaded image
        xref="x2",    # Reference x-coordinates to the x-axis
        yref="y2",    # Reference y-coordinates to the y-axis
        x=x[i], # X-coordinate of the image's center
        y=epa_y[i], # Y-coordinate of the image's center
        sizex=4,   # Width of the image in data units
        sizey=4,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.8
    )

# Format
fig.update_yaxes(
    title='Net Success Rate',
    tickformat='.0%',
    linecolor='#f0f0f0', mirror=True,
    row=1, col=1
)
fig.update_yaxes(
    title=f'Expected Margin (Net EPA)',
    tickformat='.0f',
    linecolor='#f0f0f0', mirror=True,
    row=2, col=1
)
fig.update_xaxes(
    tickfont=dict(size=8),
    tickangle=-25,
    linecolor='#f0f0f0', mirror=True
)
fig.update_layout(
    template='nfl_template',
    title=f'<b>Did We Really Get Beat That Bad?</b><br><sup>Week {WEEK}</sup>',
    height=700, width=900,
    margin_pad=0,
    margin=dict(t=75,b=60,l=40)
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.15, 
    x=1,
    align='right'
)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Did We Really Get Beat That Bad? - {net_col} - Week {WEEK}.png',
                        scale=6, width=700, height=900)

# Players

In [4]:
''' Player Stats '''

player_stats = get_player_stats(pbp_data=week_data)
player_stats = player_stats.sort_values(by='Total EPA', ascending=False)

print(player_stats.head().to_string())

                Passing player_id  Passing Plays  Passing Attempts  Passing Completions  Passing Yards  Passing TDs  Passing INTs  Passing Sacks  Passing SackYards  Passing FirstDowns  Passing Successes  Passing EPA  Passing Yds / Att  Passing Success Rate  Passing EPA / Play  Passing 1D Rate  Passing TD Rate Receiving player_id  Receiving Plays  Receiving Targets  Receiving Receptions  Receiving Yards  Receiving TDs  Receiving FirstDowns  Receiving Successes  Receiving EPA  Receiving Yds / Rec  Receiving Success Rate  Receiving EPA / Play  Receiving 1D Rate  Receiving TD Rate Rushing player_id  Rushing Plays  Rushing Attempts  Rushing Yards  Rushing TDs  Rushing FirstDowns  Rushing Successes  Rushing EPA  Rushing Yds / Att  Rushing Success Rate  Rushing EPA / Play  Rushing 1D Rate  Rushing TD Rate   player_id  Total Plays  Total Yards  Yards / Play  Total TDs   TD Rate  Total FirstDowns   1D Rate  Total EPA  EPA / Play  Total Successes  Success Rate                                    

In [ ]:
''' Player Contributions - Offense '''


for position in ['QB', 'WR', 'RB', 'TE']:

    viz_sl = player_stats.loc[(player_stats['position'] == position) & (player_stats['Total Plays'] >= 3), :]

    ## Data ##
    x = viz_sl['Total EPA'].to_numpy()
    # y = viz_sl.index.get_level_values(1)
    y = viz_sl['player_id'].tolist()
    text = viz_sl.index.get_level_values(1).tolist()
    logos = viz_sl['team_logo_espn'].tolist()
    colors = viz_sl['team_color'].tolist()
    headshots = viz_sl['headshot'].tolist()

    n_players = len(x)

    x_range = x.max() - x.min()

    ## Bar ##
    bar = px.bar(
        x=x,
        y=y,
        text=text
    )

    fig = go.Figure()

    for trace in bar.data:
        fig.add_trace(trace)

    # Logos
    for i in range(len(logos)):
        x_pos = x[i] - (x_range*.03) if x[i] < 0 else x[i] + (x_range*.03)
        fig.add_layout_image(
            # x=-0.05,
            # y=(1-((1/n_players)*.5))-((1/(n_players))*i),
            x=x_pos, y=y[i],
            # sizex=(1/n_players)*.9,
            # sizey=(1/n_players)*.9,
            sizex=1.25,
            sizey=1.25,
            xanchor='center',
            yanchor='middle',
            xref='x', 
            yref='y',
            source=logos[i],
            opacity=0.8,
            layer='below'
        )

    # Format
    fig.update_traces(marker=dict(opacity=0.7, color=colors, line=dict(width=1.5, color='black')))
    fig.update_yaxes(
        autorange='reversed',
        visible=False
    )
    fig.update_xaxes(
        title='Total EPA'
    )
    fig.update_layout(
        template='nfl_template',
        title=dict(
            text=f'<b>Offensive Player Contributions: {position}</b><br><sup>Total EPA on offensive plays, passing + receiving + rushing; min. 3 plays; Week {WEEK}</sup>',
            yref='container', y=0.97, yanchor='top'
        ),
        height=1000, width=700,
        margin=dict(t=60, r=25, b=50, l=25)
    )

    # Credits
    fig.add_annotation(
        text=f'<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
        showarrow=False,
        xref='paper',
        yref='paper',
        y=-0.05, 
        x=1,
        align='right'
    )

    fig.show()

    # Export
    if EXPORT: pio.write_image(fig, f'Offensive Contributions: {position} - Week {WEEK}.png',
                               scale=6, height=1000, width=700)